# BC COVID-19 Data by Health Authority

1. [Feature Server JSON](https://services1.arcgis.com/xeMpV7tU1t4KD3Ei/arcgis/rest/services/COVID19_Cases_by_BC_Health_Authority/FeatureServer/0?f=pjson) `id 1,2,3,4,5`

2. [Case Details](http://www.bccdc.ca/health-info/diseases-conditions/covid-19/data)

---

In [7]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape

In [3]:
r = requests.get("https://services1.arcgis.com/xeMpV7tU1t4KD3Ei/arcgis/rest/services/COVID19_Cases_by_BC_Health_Authority/FeatureServer/0/query?where=&objectIds=1%2C2%2C3%2C4%2C5&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=true&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=")
r.raise_for_status()

data = r.json()

In [4]:
gdf = gpd.GeoDataFrame.from_features(data["features"])
print(gdf.head())

                                            geometry            HA_Name
0  POLYGON ((-120.45314 52.99326, -120.42102 52.9...           Interior
1  MULTIPOLYGON (((-122.93676 49.31128, -122.9365...             Fraser
2  MULTIPOLYGON (((-123.10903 49.28623, -123.0231...  Vancouver Coastal
3  MULTIPOLYGON (((-126.66725 51.19230, -126.6705...   Vancouver Island
4  MULTIPOLYGON (((-129.76908 55.45115, -129.7657...           Northern


In [11]:
with open('../data/bc-health-authorities.geojson', 'w') as f:
    f.write(gdf.to_json())

---

2. **Case Details**

In [9]:
pd.read_csv('../data/BCCDC_COVID19_Dashboard_Case_Details.csv')

,Reported_Date,HA,Sex,Age_Group,Classification_Reported
0,2020-01-26,Out of Canada,M,40-49,Lab-diagnosed
1,2020-02-02,Vancouver Coastal,F,50-59,Lab-diagnosed
2,2020-02-05,Out of Canada,F,20-29,Lab-diagnosed
3,2020-02-05,Out of Canada,M,30-39,Lab-diagnosed
4,2020-02-11,Interior,F,30-39,Lab-diagnosed
...,...,...,...,...,...
3295,2020-07-19,Vancouver Island,M,80-89,Lab-diagnosed
3296,2020-07-19,Vancouver Island,F,60-69,Lab-diagnosed
3297,2020-07-19,Vancouver Island,F,50-59,Lab-diagnosed
3298,2020-07-19,Out of Canada,F,30-39,Lab-diagnosed


---
# BC COVID Data for LTCs

1. [long term facilities](https://www.seniorsadvocatebc.ca/quickfacts/search/%20)
2. [outbreaks](http://www.phsa.ca/current-outbreaks)

In [12]:
from bs4 import BeautifulSoup

In [13]:
re = requests.get('https://www.seniorsadvocatebc.ca/quickfacts/search/%20')
s = BeautifulSoup(re.text,'html5lib')

In [60]:
containers = s.find_all("div")

In [105]:
containers[34]

<div class="col-md-12 qf-search-record">
			<h2><a href="https://www.seniorsadvocatebc.ca/quickfacts/location/9A8ID">Acropolis Manor</a></h2>
			<p>
				Address: 1325 Summit Avenue<br/>
				City/postal: Prince Rupert V8J4C1<br/>
				Phone: (250) 622-6400			</p>
		</div>

In [83]:
# user defined exception errors
class Error(Exception):
    """Base class for other exceptions"""
    pass

class InvalidHome(Error):
    """Raised when return is not a home name or there is a duplicate"""
    pass

class InvalidAddress(Error):
    """Raised when return is not a long-term care home address"""
    pass

In [108]:
address = []
home = []
links = []


for each in containers:
    try:
         # home address
        if 'Address:' not in each.find('p').getText():
            raise InvalidAddress
        else:
            address.append(each.find('p').getText())
            print(each.find('p').getText())
            
        # home name
        if each.find('a').getText() in ['Home', '\n\t\t\t\t\t\t\t«\n\t\t\t\t\t\t', 
                                        'Data Sources', 'Visit us on Facebook', 
                                        'Email the OSA', '']:
            raise InvalidHome
        elif each.find('a').getText() in home:
            # remove duplicates (if home name already exists)
            raise InvalidHome
        else:
            home.append(each.find('a').getText())
        
        # home url
        if each.find('a').get('href') in links:
            # remove duplicates (if url already exists)
            raise InvalidHome
        else:
            links.append(each.find('a').get('href'))
        
    except AttributeError:
        pass
    except InvalidHome:
        pass
    except InvalidAddress:
        pass


				Address: 1325 Summit Avenue
				City/postal: Prince Rupert V8J4C1
				Phone: (250) 622-6400			

				Address: 1450 Hillside Ave.
				City/postal: Victoria V8T2B7
				Phone: (250) 370-5648			

				Address: 851 Boundary Road
				City/postal: Vancouver V5K4T2
				Phone: (604) 299-7567			

				Address: 2655 E Shawnigan Lake Rd
				City/postal: Shawnigan Lake V0R2W0
				Phone: (250) 743-2124			

				Address: 6764 Ayre Rd
				City/postal: Sooke V9Z1K1
				Phone: (250) 642-1750			

				Address: 1525 MacKay Crescent
				City/postal: Agassiz V0M1A3
				Phone: (604) 796-3886			

				Address: 266 Moilliet St
				City/postal: Parksville V9P1M9
				Phone: (250) 248-4331			

				Address: 4505 Valley Drive
				City/postal: Vancouver V6L2L1
				Phone: (604) 261-4292			

				Address: 2785 Ash Street
				City/postal: Vancouver V5Z1M9
				Phone: (604) 875-4111			

				Address: 635 Superior St
				City/postal: Victoria V8V1V1
				Phone: (250) 383-5447			


In [132]:
ltc = pd.DataFrame({'Home':home, 'Link':links, 'Address':address})

In [153]:
# clean and separate address info
ltc.Address = ltc.Address.str.replace('\n\t\t\t\t',' ')
ltc['city/postal'] = ltc.Address.str.extract(r'City/postal:(.*) Phone:')
ltc['phone'] = ltc.Address.str.extract(r'Phone:(.*)\t\t\t')
ltc.Address = ltc.Address.str.extract(r'Address:(.*) City/postal:')

In [154]:
ltc

,Home,Link,Address,city/postal,phone
0,Acropolis Manor,https://www.seniorsadvocatebc.ca/quickfacts/lo...,1325 Summit Avenue,Prince Rupert V8J4C1,(250) 622-6400
1,Aberdeen Hospital,https://www.seniorsadvocatebc.ca/quickfacts/lo...,1450 Hillside Ave.,Victoria V8T2B7,(250) 370-5648
2,Adanac Park Lodge,https://www.seniorsadvocatebc.ca/quickfacts/lo...,851 Boundary Road,Vancouver V5K4T2,(604) 299-7567
3,Acacia Ty Mawr,https://www.seniorsadvocatebc.ca/quickfacts/lo...,2655 E Shawnigan Lake Rd,Shawnigan Lake V0R2W0,(250) 743-2124
4,Ayre Manor,https://www.seniorsadvocatebc.ca/quickfacts/lo...,6764 Ayre Rd,Sooke V9Z1K1,(250) 642-1750
5,Agassiz Seniors Community,https://www.seniorsadvocatebc.ca/quickfacts/lo...,1525 MacKay Crescent,Agassiz V0M1A3,(604) 796-3886
6,Arrowsmith Lodge,https://www.seniorsadvocatebc.ca/quickfacts/lo...,266 Moilliet St,Parksville V9P1M9,(250) 248-4331
7,Arbutus Care Centre,https://www.seniorsadvocatebc.ca/quickfacts/lo...,4505 Valley Drive,Vancouver V6L2L1,(604) 261-4292
8,Banfield Pavilion - Vancouver General Hospital,https://www.seniorsadvocatebc.ca/quickfacts/lo...,2785 Ash Street,Vancouver V5Z1M9,(604) 875-4111
9,Beacon Hill Villa,https://www.seniorsadvocatebc.ca/quickfacts/lo...,635 Superior St,Victoria V8V1V1,(250) 383-5447
